# bTB Quick Insights and Trends
This notebook computes prevalence, top correlated features, seasonal trends, postcode hotspots, and attempts a simple logistic model for feature importance. Open and run the cells in order.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from datetime import timedelta
from geopy.distance import geodesic

In [ ]:
#Postcode gneerator for NI
def generate_ni_postcode_refined():
    """Generate realistic Northern Ireland postcodes."""
    prefixes = ['BT1', 'BT2', 'BT3', 'BT4', 'BT5', 'BT6', 'BT7', 'BT8', 'BT9', 'BT10', 
                'BT11', 'BT12', 'BT13', 'BT14', 'BT15', 'BT16', 'BT17', 'BT18', 'BT19', 'BT20',
                'BT21', 'BT22', 'BT23', 'BT24', 'BT25', 'BT26', 'BT27', 'BT28', 'BT29', 'BT30',
                'BT31', 'BT32', 'BT33', 'BT34', 'BT35', 'BT36', 'BT37', 'BT38', 'BT39', 'BT40',
                'BT41', 'BT42', 'BT43', 'BT44', 'BT45', 'BT46', 'BT47', 'BT48', 'BT49', 'BT50']
    prefix = random.choice(prefixes)
    number = random.randint(1, 9)
    letter_combo = ''.join(random.choices('ABCDEFGHIJKLMNOPQRSTUVWXYZ', k=2))
    final_number = random.randint(1, 99)
    return f"{prefix} {number}{letter_combo} {final_number}"


# Function to generate farm location with bias toward hotspots- Omagh and Banbridge
def generate_farm_location_with_hotspot_bias(tb_hotspots, min_lat, max_lat, min_lon, max_lon, hotspot_probability=0.30):
    """Generate farm location with bias toward hotspots."""
    
    if random.random() < hotspot_probability:
        # Place farm IN a hotspot
        hotspot = random.choice(tb_hotspots)
        h_lat, h_lon, h_radius = hotspot
        
        # Generate point within circular hotspot
        angle = random.uniform(0, 2 * np.pi)
        distance_km = random.uniform(0, h_radius)
        
        # Convert to lat/lon (rough approximation)
        lat_offset = (distance_km / 111.0) * np.cos(angle)
        lon_offset = (distance_km / (111.0 * np.cos(np.radians(h_lat)))) * np.sin(angle)
        
        farm_lat = h_lat + lat_offset
        farm_lon = h_lon + lon_offset
        is_in_hotspot = True
    else:
        # Place farm OUTSIDE hotspots (but still in NI)
        farm_lat = round(random.uniform(min_lat, max_lat), 6)
        farm_lon = round(random.uniform(min_lon, max_lon), 6)
        
        # Double-check it's not accidentally in a hotspot
        is_in_hotspot = False
        for h_lat, h_lon, h_radius in tb_hotspots:
            if geodesic((farm_lat, farm_lon), (h_lat, h_lon)).km < h_radius:
                is_in_hotspot = True
                break
    
    return farm_lat, farm_lon, is_in_hotspot


def generate_analytathon_data_hotspot_fixed(num_farms=1000, num_tests_per_farm=10, start_date='2020-01-01'):
    """Generates synthetic Bovine TB data with CLEAR HOTSPOTS and realistic prevalence."""

    farm_ids = [f'FARM_{i:04d}' for i in range(num_farms)]
    
    min_lat, max_lat = 54.05, 55.12
    min_lon, max_lon = -7.65, -5.83

    tb_hotspots = [
        (54.59, -7.30, 25),  # Omagh hotspot (West)
        (54.33, -5.95, 15)   # Banbridge hotspot (East)
    ]
    
    farm_details = {}
    
    for farm_id in farm_ids:
        # Generate location WITH hotspot bias
        farm_location_lat, farm_location_lon, is_in_tb_hotspot = generate_farm_location_with_hotspot_bias(
            tb_hotspots, min_lat, max_lat, min_lon, max_lon, hotspot_probability=0.30
        )
        
        farm_postcode = generate_ni_postcode_refined()
        
        # === DIFFERENTIATED FARM CHARACTERISTICS ===
        if is_in_tb_hotspot:
            proximity_to_badger_sett_km = round(random.uniform(0.5, 8.0), 1)
            badger_density_score = random.choice(['High', 'High', 'Medium'])
            biosecurity_score = random.choice([1, 1, 2, 3, 4, 5])
            herd_size = random.randint(100, 500)
            cattle_movement_in_last_year = random.randint(30, 150)
            farm_type = random.choice(['Dairy', 'Dairy', 'Mixed', 'Beef'])
            land_use_woodland_pct = random.randint(10, 35)
            has_tb_history = random.random() < 0.30
            proximity_to_other_farms_km = round(random.uniform(0.2, 5.0), 1)
        else:
            proximity_to_badger_sett_km = round(random.uniform(20.0, 120.0), 1)
            badger_density_score = random.choice(['Low', 'Low', 'Medium'])
            biosecurity_score = random.choice([2, 3, 3, 4, 5])
            herd_size = random.randint(50, 350)
            cattle_movement_in_last_year = random.randint(0, 100)
            farm_type = random.choice(['Dairy', 'Beef', 'Beef', 'Mixed'])
            land_use_woodland_pct = random.randint(0, 25)
            has_tb_history = random.random() < 0.05
            proximity_to_other_farms_km = round(random.uniform(0.5, 12.0), 1)
        
        farm_size_acres = random.randint(100, 1000)
        
        farm_details[farm_id] = {
            'Farm_Location_Lat': farm_location_lat,
            'Farm_Location_Lon': farm_location_lon,
            'Farm_Postcode': farm_postcode,
            'Herd_Size': herd_size,
            'Farm_Type': farm_type,
            'Farm_Size_Acres': farm_size_acres,
            'Biosecurity_Score': biosecurity_score,
            'Cattle_Movement_In_Last_Year': cattle_movement_in_last_year,
            'Proximity_To_Other_Farms_Km': proximity_to_other_farms_km,
            'Land_Use_Composition_Woodland_Pct': land_use_woodland_pct,
            'Has_TB_History_Farm': has_tb_history,
            'Is_In_TB_Hotspot': is_in_tb_hotspot,
            'Badger_Density_Score_Local': badger_density_score,
            'Proximity_To_Known_Badger_Sett_Km': proximity_to_badger_sett_km,
        }
    
    data = []
    
    for farm_id, details in farm_details.items():
        for i in range(num_tests_per_farm):
            test_date = pd.to_datetime(start_date) + timedelta(days=random.randint(0, 365 * 3))
            
            # === KEY FIX: DRAMATICALLY DIFFERENT BASELINES ===
            if details['Is_In_TB_Hotspot']:
                # HOTSPOTS: Start at 12% baseline (endemic region)
                prob_positive = 0.12
            else:
                # NON-HOTSPOTS: Start at 0.08% baseline (very rare)
                prob_positive = 0.0008
            
            # === ADDITIVE RISK FACTORS (REDUCED) ===
            
            # FACTOR 1: Badger proximity (reduced)
            if details['Proximity_To_Known_Badger_Sett_Km'] < 5:
                prob_positive += 0.03
            elif details['Proximity_To_Known_Badger_Sett_Km'] < 15:
                prob_positive += 0.01
            elif details['Proximity_To_Known_Badger_Sett_Km'] < 40:
                prob_positive += 0.002
            
            # FACTOR 2: Biosecurity score (reduced)
            if details['Biosecurity_Score'] == 1:
                prob_positive += 0.04
            elif details['Biosecurity_Score'] == 2:
                prob_positive += 0.02
            
            # FACTOR 3: Herd size (reduced)
            herd_size_normalized = min(details['Herd_Size'] / 500, 1.0)
            prob_positive += herd_size_normalized * 0.03
            
            # FACTOR 4: Cattle movement (reduced)
            movement_normalized = min(details['Cattle_Movement_In_Last_Year'] / 150, 1.0)
            prob_positive += movement_normalized * 0.02
            
            # FACTOR 5: Farm size (minimal)
            farm_size_normalized = min(details['Farm_Size_Acres'] / 1000, 1.0)
            prob_positive += farm_size_normalized * 0.01
            
            # FACTOR 6: Farm type (minimal)
            if details['Farm_Type'] == 'Dairy':
                prob_positive += 0.01
            
            # FACTOR 7: TB History (minimal)
            if details['Has_TB_History_Farm']:
                prob_positive += 0.015
            
            # FACTOR 8: Proximity to other farms (tiny)
            if details['Proximity_To_Other_Farms_Km'] < 2:
                prob_positive += 0.008
            elif details['Proximity_To_Other_Farms_Km'] < 5:
                prob_positive += 0.002
            
            # FACTOR 9: Seasonal variation (minimal)
            test_season = pd.to_datetime(test_date).strftime('%B')
            if test_season in ['September', 'October', 'November', 'December']:
                prob_positive += 0.002
            elif test_season in ['June', 'July', 'August']:
                prob_positive -= 0.001
            
            # FACTOR 10: Animal age (tiny)
            animal_age_months = random.randint(12, 180)
            if animal_age_months > 120:
                prob_positive += 0.001
            
            # FACTOR 11: Animal origin (tiny)
            origin_type = random.choice(['Born_On_Farm', 'Born_On_Farm', 'Purchased_From_Other_Farm'])
            if origin_type == 'Purchased_From_Other_Farm':
                prob_positive += 0.001
            
            # Add stochastic noise and clip to valid range
            prob_positive = np.clip(prob_positive + np.random.normal(0, 0.002), 0.00001, 0.60)
            
            test_result = 'Positive' if random.random() < prob_positive else 'Negative'
            
            animal_id = f'{farm_id}_ANIMAL_{random.randint(1, details["Herd_Size"]):03d}'
            sex = random.choice(['Male', 'Female'])
            breed = random.choice(['Holstein', 'Jersey', 'Angus', 'Hereford', 'Charolais', 'Limousin'])
            
            days_since_last_test = random.randint(90, 730)
            avg_rainfall_3m_prior_mm = random.randint(100, 400)
            avg_temperature_3m_prior_c = round(random.uniform(4.0, 18.0), 1)

            data.append({
                'Farm_ID': farm_id,
                'Farm_Location_Lat': details['Farm_Location_Lat'],
                'Farm_Location_Lon': details['Farm_Location_Lon'],
                'Farm_Postcode': details['Farm_Postcode'],
                'Herd_Size': details['Herd_Size'],
                'Farm_Type': details['Farm_Type'],
                'Farm_Size_Acres': details['Farm_Size_Acres'],
                'Biosecurity_Score': details['Biosecurity_Score'],
                'Cattle_Movement_In_Last_Year': details['Cattle_Movement_In_Last_Year'],
                'Proximity_To_Other_Farms_Km': details['Proximity_To_Other_Farms_Km'],
                'Land_Use_Composition_Woodland_Pct': details['Land_Use_Composition_Woodland_Pct'],
                'Animal_ID': animal_id,
                'Animal_Age_Months': animal_age_months,
                'Breed': breed,
                'Sex': sex,
                'Origin_Type': origin_type,
                'Test_Date': test_date,
                'Test_Result': test_result,
                'Has_TB_History_Farm': details['Has_TB_History_Farm'],
                'Is_In_TB_Hotspot': details['Is_In_TB_Hotspot'],
                'Avg_Rainfall_3M_Prior_MM': avg_rainfall_3m_prior_mm,
                'Avg_Temperature_3M_Prior_C': avg_temperature_3m_prior_c,
                'Badger_Density_Score_Local': details['Badger_Density_Score_Local'],
                'Proximity_To_Known_Badger_Sett_Km': details['Proximity_To_Known_Badger_Sett_Km'],
                'Days_Since_Last_Test': days_since_last_test,
                'Test_Season': test_season,
            })
    
    df = pd.DataFrame(data)
    return df


if __name__ == "__main__":
    df_analytathon = generate_analytathon_data_hotspot_fixed(num_farms=1000, num_tests_per_farm=10)
    
    print(f"Total records: {len(df_analytathon)}")
    print(f"Positive tests: {(df_analytathon['Test_Result'] == 'Positive').sum()}")
    print(f"Overall TB prevalence: {(df_analytathon['Test_Result'] == 'Positive').mean():.2%}")
    
    hotspot_df = df_analytathon[df_analytathon['Is_In_TB_Hotspot'] == True]
    non_hotspot_df = df_analytathon[df_analytathon['Is_In_TB_Hotspot'] == False]
    
    print(f"\n=== HOTSPOT ANALYSIS ===")
    print(f"Farms in hotspots: {hotspot_df['Farm_ID'].nunique()}")
    print(f"TB prevalence (hotspots): {(hotspot_df['Test_Result'] == 'Positive').mean():.1%}")
    print(f"Positive cases (hotspots): {(hotspot_df['Test_Result'] == 'Positive').sum()}")
    
    print(f"\nFarms outside hotspots: {non_hotspot_df['Farm_ID'].nunique()}")
    print(f"TB prevalence (non-hotspots): {(non_hotspot_df['Test_Result'] == 'Positive').mean():.3%}")
    print(f"Positive cases (non-hotspots): {(non_hotspot_df['Test_Result'] == 'Positive').sum()}")
    
    if (non_hotspot_df['Test_Result'] == 'Positive').mean() > 0:
        risk_ratio = (hotspot_df['Test_Result'] == 'Positive').mean() / (non_hotspot_df['Test_Result'] == 'Positive').mean()
        print(f"\n✓ Risk Ratio (Hotspot vs Non-Hotspot): {risk_ratio:.1f}x")
    
    df_analytathon.to_csv('farm_data.csv', index=False)
    print("\n✓ Data saved to 'farm_data.csv'")

Total records: 10000
Positive tests: 1059
Overall TB prevalence: 10.59%

=== HOTSPOT ANALYSIS ===
Farms in hotspots: 415
TB prevalence (hotspots): 20.6%
Positive cases (hotspots): 855

Farms outside hotspots: 585
TB prevalence (non-hotspots): 3.487%
Positive cases (non-hotspots): 204

✓ Risk Ratio (Hotspot vs Non-Hotspot): 5.9x

✓ Data saved to 'farm_data.csv'


In [ ]:
#df.drop(columns=['Is_In_TB_Hotspot'], inplace=True)
#df.to_csv('farm_data.csv', index=False)